In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [4]:
# Load datasets with dtype specifications to handle mixed types
train = pd.read_csv('/content/train.csv', dtype={'StateHoliday': str})
test = pd.read_csv('/content/test.csv', dtype={'StateHoliday': str})
store = pd.read_csv('/content/store.csv')

In [ ]:
train.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday'],
      dtype='object')

In [ ]:
test.columns

Index(['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday'],
      dtype='object')

In [ ]:
store.columns

Index(['Store', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

# Handle missing values

In [5]:
# 1. CompetitionDistance
median_competition_distance = store['CompetitionDistance'].median()
store['CompetitionDistance'].fillna(median_competition_distance, inplace=True)

In [6]:
# 2. CompetitionOpenSinceMonth
mode_competition_open_since_month = store['CompetitionOpenSinceMonth'].mode()[0]
store['CompetitionOpenSinceMonth'].fillna(mode_competition_open_since_month, inplace=True)

In [7]:
# 3. CompetitionOpenSinceYear
mode_competition_open_since_year = store['CompetitionOpenSinceYear'].mode()[0]
store['CompetitionOpenSinceYear'].fillna(mode_competition_open_since_year, inplace=True)

In [8]:
# 4. Promo2SinceWeek
mode_promo2_since_week = store['Promo2SinceWeek'].mode()[0]
store['Promo2SinceWeek'].fillna(mode_promo2_since_week, inplace=True)

In [9]:
# 5. Promo2SinceYear
mode_promo2_since_year = store['Promo2SinceYear'].mode()[0]
store['Promo2SinceYear'].fillna(mode_promo2_since_year, inplace=True)

In [10]:
# 6. PromoInterval
store['PromoInterval'].fillna('None', inplace=True)

For the missing values in the Open column of the test dataset, we can impute them based on some logical assumptions or patterns observed in the data. One common approach is to assume that stores are open if there is no explicit indication that they are closed.

In [11]:
test['Open'].fillna(1, inplace=True)
test.isnull().sum()

Id               0
Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [ ]:
store.isnull().sum()

Store                        0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64

In [ ]:
train.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [ ]:
test.isnull().sum()

Id               0
Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [ ]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,14.0,2011.0,None
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,14.0,2011.0,None
4,5,a,a,29910.0,4.0,2015.0,0,14.0,2011.0,None


In [12]:
# Convert 'Date' column to datetime
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [ ]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,14.0,2011.0,None
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,14.0,2011.0,None
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,14.0,2011.0,None


In [ ]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,14.0,2011.0,None
1,2,3,4,2015-09-17,1.0,1,0,0,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,7,4,2015-09-17,1.0,1,0,0,a,c,24000.0,4.0,2013.0,0,14.0,2011.0,None
3,4,8,4,2015-09-17,1.0,1,0,0,a,a,7520.0,10.0,2014.0,0,14.0,2011.0,None
4,5,9,4,2015-09-17,1.0,1,0,0,a,c,2030.0,8.0,2000.0,0,14.0,2011.0,None


In [13]:
def extract_date_features(df):
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['IsWeekend'] = df['DayOfWeek'] >= 5
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week
    df['Quarter'] = df['Date'].dt.quarter
    df['IsMonthStart'] = df['Date'].dt.is_month_start
    df['IsMonthEnd'] = df['Date'].dt.is_month_end
    df['IsQuarterStart'] = df['Date'].dt.is_quarter_start
    df['IsQuarterEnd'] = df['Date'].dt.is_quarter_end

    # Extract holiday dates
    holiday_dates = df[df['StateHoliday'] != '0']['Date'].sort_values().unique()

    # Convert the array to a pandas Series for vectorized operations
    holiday_series = pd.Series(holiday_dates)

    # Initialize arrays for results
    days_to_holiday = np.full(len(df), np.nan)
    days_after_holiday = np.full(len(df), np.nan)

    # Use searchsorted to find the indices of the next and previous holidays
    idx_next_holiday = np.searchsorted(holiday_series, df['Date'], side='left')
    idx_prev_holiday = np.searchsorted(holiday_series, df['Date'], side='right') - 1

    # Calculate days to the next holiday
    valid_next_holiday = idx_next_holiday < len(holiday_series)
    days_to_holiday[valid_next_holiday] = (holiday_series.iloc[idx_next_holiday[valid_next_holiday]].values - df['Date'][valid_next_holiday].values) / np.timedelta64(1, 'D')

    # Calculate days after the last holiday
    valid_prev_holiday = idx_prev_holiday >= 0
    days_after_holiday[valid_prev_holiday] = (df['Date'][valid_prev_holiday].values - holiday_series.iloc[idx_prev_holiday[valid_prev_holiday]].values) / np.timedelta64(1, 'D')

    # Assign results back to the DataFrame
    df['DaysToHoliday'] = days_to_holiday
    df['DaysAfterHoliday'] = days_after_holiday

    return df

In [14]:
extract_date_features(train)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,Day,IsWeekend,WeekOfYear,Quarter,IsMonthStart,IsMonthEnd,IsQuarterStart,IsQuarterEnd,DaysToHoliday,DaysAfterHoliday
0,1,4,2015-07-31,5263,555,1,1,0,1,2015,...,31,False,31,3,False,True,False,False,NaN,57.0
1,2,4,2015-07-31,6064,625,1,1,0,1,2015,...,31,False,31,3,False,True,False,False,NaN,57.0
2,3,4,2015-07-31,8314,821,1,1,0,1,2015,...,31,False,31,3,False,True,False,False,NaN,57.0
3,4,4,2015-07-31,13995,1498,1,1,0,1,2015,...,31,False,31,3,False,True,False,False,NaN,57.0
4,5,4,2015-07-31,4822,559,1,1,0,1,2015,...,31,False,31,3,False,True,False,False,NaN,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,1,2013-01-01,0,0,0,0,a,1,2013,...,1,False,1,1,True,False,True,False,0.0,0.0
1017205,1112,1,2013-01-01,0,0,0,0,a,1,2013,...,1,False,1,1,True,False,True,False,0.0,0.0
1017206,1113,1,2013-01-01,0,0,0,0,a,1,2013,...,1,False,1,1,True,False,True,False,0.0,0.0
1017207,1114,1,2013-01-01,0,0,0,0,a,1,2013,...,1,False,1,1,True,False,True,False,0.0,0.0


In [15]:
extract_date_features(test)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,IsWeekend,WeekOfYear,Quarter,IsMonthStart,IsMonthEnd,IsQuarterStart,IsQuarterEnd,DaysToHoliday,DaysAfterHoliday
0,1,1,3,2015-09-17,1.0,1,0,0,2015,9,17,False,38,3,False,False,False,False,NaN,33.0
1,2,3,3,2015-09-17,1.0,1,0,0,2015,9,17,False,38,3,False,False,False,False,NaN,33.0
2,3,7,3,2015-09-17,1.0,1,0,0,2015,9,17,False,38,3,False,False,False,False,NaN,33.0
3,4,8,3,2015-09-17,1.0,1,0,0,2015,9,17,False,38,3,False,False,False,False,NaN,33.0
4,5,9,3,2015-09-17,1.0,1,0,0,2015,9,17,False,38,3,False,False,False,False,NaN,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41083,41084,1111,5,2015-08-01,1.0,0,0,0,2015,8,1,True,31,3,True,False,False,False,14.0,NaN
41084,41085,1112,5,2015-08-01,1.0,0,0,0,2015,8,1,True,31,3,True,False,False,False,14.0,NaN
41085,41086,1113,5,2015-08-01,1.0,0,0,0,2015,8,1,True,31,3,True,False,False,False,14.0,NaN
41086,41087,1114,5,2015-08-01,1.0,0,0,0,2015,8,1,True,31,3,True,False,False,False,14.0,NaN


In [16]:
# Merge store information
train = train.merge(store, on='Store', how='left')
test = test.merge(store, on='Store', how='left')

In [ ]:
train.dtypes

Store                                 int64
DayOfWeek                             int32
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                  int64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
Year                                  int32
Month                                 int32
Day                                   int32
IsWeekend                              bool
WeekOfYear                           UInt32
Quarter                               int32
IsMonthStart                           bool
IsMonthEnd                             bool
IsQuarterStart                         bool
IsQuarterEnd                           bool
DaysToHoliday                       float64
DaysAfterHoliday                    float64
StoreType                            object
Assortment                      

In [ ]:
test.dtypes

Id                                    int64
Store                                 int64
DayOfWeek                             int32
Date                         datetime64[ns]
Open                                float64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
Year                                  int32
Month                                 int32
Day                                   int32
IsWeekend                              bool
WeekOfYear                           UInt32
Quarter                               int32
IsMonthStart                           bool
IsMonthEnd                             bool
IsQuarterStart                         bool
IsQuarterEnd                           bool
DaysToHoliday                       float64
DaysAfterHoliday                    float64
StoreType                            object
Assortment                           object
CompetitionDistance             

In [17]:
def preprocess_data(df):
  # Map values
  mapping = {'0': 1, 'a': 0, 'b': 0, 'c': 0}
  df['StateHoliday'] = df['StateHoliday'].replace(mapping)
  columns=['StoreType', 'Assortment', 'PromoInterval']

  # Convert categorical features to numeric using one-hot encoding
  dataset = pd.get_dummies(df, columns=columns, drop_first=True)
  return dataset

In [18]:
train = preprocess_data(train)

In [19]:
test = preprocess_data(test)

In [ ]:
train.dtypes

Store                                      int64
DayOfWeek                                  int32
Date                              datetime64[ns]
Sales                                      int64
Customers                                  int64
Open                                       int64
Promo                                      int64
StateHoliday                               int64
SchoolHoliday                              int64
Year                                       int32
Month                                      int32
Day                                        int32
IsWeekend                                   bool
WeekOfYear                                UInt32
Quarter                                    int32
IsMonthStart                                bool
IsMonthEnd                                  bool
IsQuarterStart                              bool
IsQuarterEnd                                bool
DaysToHoliday                            float64
DaysAfterHoliday    

In [ ]:
test.dtypes

Id                                         int64
Store                                      int64
DayOfWeek                                  int32
Date                              datetime64[ns]
Open                                     float64
Promo                                      int64
StateHoliday                               int64
SchoolHoliday                              int64
Year                                       int32
Month                                      int32
Day                                        int32
IsWeekend                                   bool
WeekOfYear                                UInt32
Quarter                                    int32
IsMonthStart                                bool
IsMonthEnd                                  bool
IsQuarterStart                              bool
IsQuarterEnd                                bool
DaysToHoliday                            float64
DaysAfterHoliday                         float64
CompetitionDistance 

In [20]:
# Calculate the correlation matrix
correlation_matrix = train.corr()

In [21]:
correlation_matrix['Sales']

Store                             0.005126
DayOfWeek                        -0.462125
Date                              0.041904
Sales                             1.000000
Customers                         0.894711
Open                              0.678472
Promo                             0.452345
StateHoliday                      0.254216
SchoolHoliday                     0.085124
Year                              0.023519
Month                             0.048768
Day                              -0.011612
IsWeekend                        -0.450152
WeekOfYear                        0.052946
Quarter                           0.043984
IsMonthStart                     -0.053450
IsMonthEnd                        0.047112
IsQuarterStart                   -0.013222
IsQuarterEnd                      0.027899
DaysToHoliday                     0.015372
DaysAfterHoliday                  0.055244
CompetitionDistance              -0.018869
CompetitionOpenSinceMonth        -0.023226
Competition

In [22]:
# Set the threshold value
threshold = 0.05

In [23]:
# Filter the columns based on the threshold
filtered_columns = correlation_matrix['Sales'][abs(correlation_matrix['Sales']) >= threshold].sort_values(ascending=False)

In [24]:
filtered_columns

Sales                             1.000000
Customers                         0.894711
Open                              0.678472
Promo                             0.452345
StateHoliday                      0.254216
StoreType_b                       0.139940
PromoInterval_None                0.091040
SchoolHoliday                     0.085124
Assortment_c                      0.068772
Assortment_b                      0.065473
DaysAfterHoliday                  0.055244
WeekOfYear                        0.052946
PromoInterval_Mar,Jun,Sept,Dec   -0.053267
IsMonthStart                     -0.053450
Promo2                           -0.091040
IsWeekend                        -0.450152
DayOfWeek                        -0.462125
Name: Sales, dtype: float64

In [25]:
# Extract the column names
filtered_column_names = filtered_columns.index.tolist()
filtered_column_names[1:]

['Customers',
 'Open',
 'Promo',
 'StateHoliday',
 'StoreType_b',
 'PromoInterval_None',
 'SchoolHoliday',
 'Assortment_c',
 'Assortment_b',
 'DaysAfterHoliday',
 'WeekOfYear',
 'PromoInterval_Mar,Jun,Sept,Dec',
 'IsMonthStart',
 'Promo2',
 'IsWeekend',
 'DayOfWeek']

In [26]:
from sklearn.preprocessing import StandardScaler

In [27]:
# Separate target variable
X_test = test[filtered_column_names[2:]]
# y_test = test['Sales']
# Separate target variable
X_train = train[filtered_column_names[2:]]
y_train = train['Sales']

# Scale all input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # Fit and transform X_train
# Scale all input features
X_test_scaled = scaler.transform(X_test) # Transform X_test using the fitted scaler

In [ ]:
y_train

0           5263
1           6064
2           8314
3          13995
4           4822
           ...  
1017204        0
1017205        0
1017206        0
1017207        0
1017208        0
Name: Sales, Length: 1017209, dtype: int64

In [28]:
X_train

,Open,Promo,StateHoliday,StoreType_b,PromoInterval_None,SchoolHoliday,Assortment_c,Assortment_b,DaysAfterHoliday,WeekOfYear,"PromoInterval_Mar,Jun,Sept,Dec",IsMonthStart,Promo2,IsWeekend,DayOfWeek
0,1,1,1,False,True,1,False,False,57.0,31,False,False,0,False,4
1,1,1,1,False,False,1,False,False,57.0,31,False,False,1,False,4
2,1,1,1,False,False,1,False,False,57.0,31,False,False,1,False,4
3,1,1,1,False,True,1,True,False,57.0,31,False,False,0,False,4
4,1,1,1,False,True,1,False,False,57.0,31,False,False,0,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,0,0,0,False,False,1,False,False,0.0,1,False,True,1,False,1
1017205,0,0,0,False,True,1,True,False,0.0,1,False,True,0,False,1
1017206,0,0,0,False,True,1,True,False,0.0,1,False,True,0,False,1
1017207,0,0,0,False,True,1,True,False,0.0,1,False,True,0,False,1


In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

# Create pipeline
pipeline = Pipeline(steps=[('regressor', RandomForestRegressor(n_estimators=100, random_state=42))])

In [31]:
model = pipeline.fit(X_train_scaled, y_train)

In [32]:
model

Pipeline(steps=[('regressor', RandomForestRegressor(random_state=42))])

In [30]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

# Define the model

# Perform cross-validation and get the scores
cv_scores = cross_val_score(pipeline, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')

# Convert negative MSE to positive and take the square root to get RMSE
rmse_scores = (-cv_scores)**0.5

# Print the scores and the mean RMSE
print("Cross-validation RMSE scores:", rmse_scores)
print("Mean RMSE:", rmse_scores.mean())

Cross-validation RMSE scores: [2390.0197901  2487.34799267 2353.49090255 2506.34545231 2409.21156865]
Mean RMSE: 2429.2831412552277
